# Centralni granični teorem

In [1]:
import numpy as np; from numpy import *
import numpy.random as rnd;
import matplotlib.pyplot as plt
import ipywidgets as widgets; from ipywidgets import *
from shared import *
from gauss import *
from clt import *
from matplotlib import rc
rc('text', usetex=True)
rc('font', size=14)

## Binomna raspodjela

In [2]:
def visualize_binomial(n, approx):
    plot_binomial(n ,approx)
    plt.plot()
interact(visualize_binomial,
         n=IntSlider(1,1,100),
         approx=['Auto', 'True', 'False']);

interactive(children=(IntSlider(value=1, description='n', min=1), Dropdown(description='approx', options=('Aut…

## Degeneracija normalne raspodjele u delta funkciju

In [3]:
def visualize_degenerate(n):
    plot_degenerate([n])
    plt.show()

interact(visualize_degenerate, n=IntSlider(1, min=1, max=200));

interactive(children=(IntSlider(value=1, description='n', max=200, min=1), Output()), _dom_classes=('widget-in…

## Višestruka konvolucija signala

In [4]:
def visualize_convolution(n, tip):
    # Ulazna funkcija
    plt.subplot(2,1,1)
    f = gen_fun(tip)
    plot_fun(f)
    
    # Konvolucija
    plt.subplot(2,1,2)
    plot_convolution(f, n)
    plt.tight_layout(); plt.show()
    
# Napomena: Povećavanje n iznad preporučenog maksimuma može dovesti do
# numeričkih problema
interact(visualize_convolution, n=IntSlider(1, min=1, max=20),
         tip=['rect', 'noise_uniform', 'noise_exp', 'exp', 'exp2']);

interactive(children=(IntSlider(value=1, description='n', max=20, min=1), Dropdown(description='tip', options=…

## Obojeno uzorkovanje

In [5]:
"""
    Izvlačenje uzoraka iz populacije i grafički prikaz.
    
    N - Veličina populacije
    N1 - Veličina uzorka
"""
def plot_uzorkovanje(Ns, N=1e4, tip='uniform',
                pop=True, hist_pop=True, hist=False,
                oboji=True, randomiziraj=True):

    N1 = int(N / Ns) # Veličina jednog uzorka
    N = Ns * N1 # Radi izbjegavanja problema sa dimenzijama matrica
    X = rnd.uniform(100,1, size=N) # Generisanje populacije

    if not randomiziraj:
        X = sort(X)
        
    # Formiranje slučajnih uzoraka
    indexi_uzoraka = rnd.choice(range(N), (Ns, N1), replace=False)
    uzorci = X.take(indexi_uzoraka)
    
    i = 1 # indeks subplot-a [*] 
    plt.figure()
    
    # Prikaz populacije zajedno sa slučajnim uzorcima
    if pop:
        plt.subplot(3,1,i); i += 1 # [*] koristi se ovdje
        if oboji:
            for i in range(Ns):
                plt.scatter(indexi_uzoraka[i], uzorci[i])
        else:
            plt.scatter(range(N), X)
        
    
    # Prikaz histograma populacije
    if hist_pop:
        plt.subplot(3,1,2); i += 1 # [*] koristi se ovdje
        if oboji:
            plt.hist(uzorci.T, stacked=True, density=True)
        else:
            plt.hist(X, bins=30)
            
    # Prikaz histograma srednjih vrijednosti uzoraka
    if hist:
        # Niz srednjih vrijednosti uzoraka
        S = mean(uzorci, axis=1)
        plt.subplot(3,1,i) # [*] koristi se ovdje
        f, x, _ = plt.hist(S, bins=20, density=True)
        x = (x[:-1] + x[1:]) / 2
        try:
            plt.plot(x, approx_gaussian(x, f)) # Normalna aproksimacija
        except: plt.show()
        
    plt.tight_layout()
    plt.show()

In [6]:
interact(plot_uzorkovanje,
         Ns=IntSlider(20, 1,1000),
         N=IntSlider(10000, 1,1e5),
         tip=['uniform', 'normal', 'exponential'],
         oboji=False,
         randomiziraj=True,
         pop=fixed(True),
         hist_pop=fixed(True),
         hist=fixed(True)
        );

interactive(children=(IntSlider(value=20, description='Ns', max=1000, min=1), IntSlider(value=10000, descripti…

## Bonus

In [7]:
# Fourierova transformacija
import scipy.fft as fft

def fft_n(n, tip):
    x = gen_fun(tip)
    w = arange(-50, 50)
    
    X = sinc(w)
    X = abs(fft.fftshift(fft.fft(x)))
    Y = X**n / 100
    Y_approx = approx_gaussian(w, Y)

    plt.subplot(2,1,1)
    plt.plot(arange(-50, 50), x)
    plt.subplot(2,1,2)
    plt.plot(w, Y_approx, label='Aproksimacija')
    plt.plot(w, abs(Y), label='F.T.')
    plt.legend(bbox_to_anchor=(0.55,1), framealpha=1); plt.show()
    
interact(fft_n,
         n=widgets.IntSlider(2, 2, 10, step=2),
         tip=['rect', 'noise_uniform', 'noise_exp', 'exp']
         );

interactive(children=(IntSlider(value=2, description='n', max=10, min=2, step=2), Dropdown(description='tip', …